# Running a Power Flow

We load the simple example network from the create_network tutorial from the pandapower.networks module:

In [1]:
import pandapower as pp
import pandapower.networks

net = pandapower.networks.example_simple()
net

This pandapower network includes the following parameter tables:
   - ext_grid (1 elements)
   - trafo (1 elements)
   - load (1 elements)
   - shunt (1 elements)
   - gen (1 elements)
   - bus (7 elements)
   - line (4 elements)
   - sgen (1 elements)
   - switch (8 elements)

## Run a Power Flow and Access Results

Runing a loadflow adds seperate result table with the prefix 'res_':

In [2]:
pp.runpp(net)

In [3]:
net

This pandapower network includes the following parameter tables:
   - ext_grid (1 elements)
   - trafo (1 elements)
   - load (1 elements)
   - shunt (1 elements)
   - gen (1 elements)
   - bus (7 elements)
   - line (4 elements)
   - sgen (1 elements)
   - switch (8 elements)
 and the following results tables:
   - res_bus (7 elements)
   - res_trafo (1 elements)
   - res_gen (1 elements)
   - res_shunt (1 elements)
   - res_sgen (1 elements)
   - res_load (1 elements)
   - res_ext_grid (1 elements)
   - res_line (4 elements)

These results tables are pandas datafarmes with the same index as the element table. For example, the bus table contains all bus voltages and summed bus power injections:

In [4]:
 net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,0.000000,6727.268204,7262.701273
1,1.020842,0.032006,0.000000,0.000000
2,1.020842,0.032006,0.000000,-1000.434322
3,1.024513,1.683724,0.000000,0.000000
4,1.024513,1.683724,0.000000,0.000000
5,1.030000,1.748309,-6000.000000,-3513.228774
6,1.023156,1.833115,-800.000000,2900.000000


We can now use pandas functionality to analyse the loadflow results, for example to get the minimum voltage in the medium voltage level:

In [5]:
net.res_bus[net.bus.vn_kv==20.].vm_pu.min()

1.0231555578412688

or the maxium voltage at a bus with load or generation:

In [6]:
load_or_generation_buses = set(net.load.bus.values) | set(net.sgen.bus.values) | set(net.gen.bus.values)
net.res_bus.vm_pu.loc[load_or_generation_buses].max()

1.03

For more on how to use pandas for data analysis in pandapower, see the tutorial on [data analysis](data_analysis.ipynb).

## Result tables

Each element (except the switch) has its own result table with results tailored to the specific element. Here, we just show each table. For parameters definitions, see the documentation of the datastructure.

In [7]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,0.000000,6727.268204,7262.701273
1,1.020842,0.032006,0.000000,0.000000
2,1.020842,0.032006,0.000000,-1000.434322
3,1.024513,1.683724,0.000000,0.000000
4,1.024513,1.683724,0.000000,0.000000
5,1.030000,1.748309,-6000.000000,-3513.228774
6,1.023156,1.833115,-800.000000,2900.000000


In [8]:
net.res_ext_grid

,p_kw,q_kvar
0,6727.268204,7262.701273


In [9]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_ka,loading_percent
0,-6727.268204,-7262.701331,6.730354e+03,1.570346e+03,3.086061,-5692.355559,0.050941,8.663409
1,-5972.020871,-3572.601663,6.000000e+03,3.517662e+03,27.979108,-54.940094,0.196084,46.575820
2,0.000024,-4.432793,9.638904e-08,6.041912e-08,0.000024,-4.432793,0.000124,0.059160
3,800.000000,-2900.000000,-7.936175e+02,2.805748e+03,6.382452,-94.251942,0.084877,20.160899


In [10]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354267,-569.911445,6765.638421,766.853612,35.284154,196.942168,0.034728,0.191855,26.584187


In [11]:
net.res_load

,p_kw,q_kvar
0,1200.0,2400.0


In [12]:
net.res_sgen

,p_kw,q_kvar
0,-2000.0,500.0


In [13]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3513.228774,1.748309,1.03


In [14]:
net.res_shunt

,p_kw,q_kvar,vm_pu
0,0.0,-1000.434322,1.020842


## Voltage Angles and Initialization

Maybe you wondered why even though there is a voltage angle of 50 degrees defined for the external grid: 

In [15]:
net.ext_grid.va_degree

0    50.0
Name: va_degree, dtype: float64

and a shift of 150° over the HV/MV transformer:

In [16]:
net.trafo.shift_degree

0    150.0
Name: shift_degree, dtype: float64

the voltage angles are all close to zero:

In [17]:
pp.runpp(net)
net.res_bus.va_degree

0    0.000000
1    0.032006
2    0.032006
3    1.683724
4    1.683724
5    1.748309
6    1.833115
Name: va_degree, dtype: float64

That is because the standard parameter for calculate_voltage_angles is False, which means voltage angles at external grids and transformer shifts are ignored by default. In a radial network, the absolute voltage angle shifts do not have an influence on the power flow, which is why they are disabled by default. In meshed networks however, where multiple external grids are galvanically coupled, it is always necessary to calculate the voltage angles.

Suppose we want to calculate the correct voltage angles and set calculate_voltage_angles to True:

In [18]:
pp.runpp(net, calculate_voltage_angles=True)

LoadflowNotConverged: Loadflow did not converge!

Now the power flow does not converge. This can happen with large angle shifts. The solution is to use a initialization with a DC loadflow instead of a flat start, which is default behaviour:

In [19]:
pp.runpp(net, calculate_voltage_angles=True, init="dc")

Now, we can see that all voltage angles are correctly calculated:

In [20]:
net.res_bus.va_degree

0    50.000000
1    50.032006
2    50.032006
3   -98.316276
4   -98.316276
5   -98.251691
6   -98.166885
Name: va_degree, dtype: float64

If we already have a solution, we can also initialize the loadflow with the voltage values from the last loadflow:

In [21]:
pp.runpp(net, calculate_voltage_angles=True, init="results")
net.res_bus.va_degree

0    50.000000
1    50.032006
2    50.032006
3   -98.316276
4   -98.316276
5   -98.251691
6   -98.166885
Name: va_degree, dtype: float64

The power flow converges and yields correct results where a flat start power flow would have failed.

Initializing with previous results can save convergence time in cases where multiple power flows with simliar input parameters are carried out consecutively, such as in quasi-static time series simulations.

## Transformer Model

The parameter "trafo_model" can be used to switch between a 'pi' and a 't' transformer model:

In [22]:
pp.runpp(net, trafo_model="t")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354267,-569.911445,6765.638421,766.853612,35.284154,196.942168,0.034728,0.191855,26.584187


In [23]:
pp.runpp(net, trafo_model="pi")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.348663,-569.72374,6765.639204,766.663363,35.290541,196.939623,0.034728,0.191854,26.584104


For a definition of the different transformer model see the power flow model documentation of the transformer element.

## Transformer Loading

The transformer loading can either be calculated in relation to the rated current:

In [24]:
pp.runpp(net, trafo_loading="current")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354267,-569.911445,6765.638421,766.853612,35.284154,196.942168,0.034728,0.191855,26.584187


or to the rated power of the transformer:

In [25]:
pp.runpp(net, trafo_loading="power")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354267,-569.911445,6765.638421,766.853612,35.284154,196.942168,0.034728,0.191855,27.235837


The transformer loading does not have an influence on other power flow results besides the loading_percent parameter.

## Generator Reactive Power Limits

The generator has reactive power limits of -3000...3000 kvar:

In [26]:
net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type
0,generator,5,-6000.0,1.03,NaN,-3000.0,3000.0,1.0,True,None


which are however exceeded in the power flow results, because the enforce_q_lims option defaults to False:

In [27]:
pp.runpp(net)
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3513.228774,1.748309,1.03


If the enforce_q_lims parameter is set to True, the reactive power limit is complied with, while the voltage deviates from the voltage set point of the generator:

In [28]:
pp.runpp(net, enforce_q_lims=True)
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3000.0,1.774554,1.027434


If you want to know what to do when a power flow does not converge, continue with the [diagnostic tutorial](diagnostic.ipynb).